In [1]:
import numpy as np
import pandas as pd
import bqapi as bq
sess= bq.Session()

import pyrefdata as pr

In [2]:
## get prices of constituents
price_data = pr.get_historical_data(sess, 'index:indu', 'px_last', '01-01-2014', '12-13-2016').unstack().T.loc['px_last']

In [3]:
## get prices of index
index_prices = pr.get_historical_data(sess, 'indu index', 'px_last', '01-01-2014', '12-13-2016').unstack().T.loc['px_last']

In [4]:
returns = price_data.pct_change()[1:]
returns_clean = returns.dropna(axis=1)

corr_mat = np.corrcoef(returns_clean.T)

In [5]:
from bqplot import pyplot as plt
from bqplot import GridHeatMap, Figure, ColorScale, ColorAxis

from ipywidgets import HBox

In [6]:
def event_handler(name, value):
    if brush.selected is None or len(brush.selected) == 0:
        _corr = np.corrcoef(returns_clean).T
    else:
        sel = [pd.to_datetime(s) for s in brush.selected]
        _corr = np.corrcoef(returns_clean.loc[sel[0]:sel[1], :]).T
    corr_mat.color = _corr

In [7]:
sc_col = ColorScale(scheme='Reds', min=0.0, max=1.0)
ax_col = ColorAxis(scale=sc_col, label='Correlation')

corr_mat = GridHeatMap(color=np.corrcoef(returns_clean.iloc[:, 1:].T),
                       scales={'color': sc_col})
corr_fig = Figure(marks=[corr_mat], axes=[ax_col], title='DOW Jones Correlation Matrix')

plt.figure(title='DOW Jones IA Price Series')
price_line = plt.plot(index_prices.index.values, index_prices.values)
brush = plt.int_selector(event_handler)
price_fig = plt.current_figure()

HBox([price_fig, corr_fig])

In [8]:
corr_mat.color

array([[ 1.        ,  0.00134909, -0.13378207, ..., -0.19492845,
         0.39206648, -0.18580654],
       [ 0.00134909,  1.        , -0.06007245, ...,  0.16691401,
        -0.10203542, -0.20367494],
       [-0.13378207, -0.06007245,  1.        , ..., -0.17991704,
        -0.09583758, -0.38208258],
       ..., 
       [-0.19492845,  0.16691401, -0.17991704, ...,  1.        ,
        -0.08393485,  0.0021062 ],
       [ 0.39206648, -0.10203542, -0.09583758, ..., -0.08393485,
         1.        , -0.04516078],
       [-0.18580654, -0.20367494, -0.38208258, ...,  0.0021062 ,
        -0.04516078,  1.        ]])

In [9]:
corr_mat.color

array([[ 1.        , -0.0693465 ,  0.08457293, ...,  0.06864262,
        -0.23728171,  0.24414068],
       [-0.0693465 ,  1.        ,  0.15926824, ...,  0.25789105,
         0.02631703,  0.24682541],
       [ 0.08457293,  0.15926824,  1.        , ...,  0.05202172,
         0.16257918,  0.30441038],
       ..., 
       [ 0.06864262,  0.25789105,  0.05202172, ...,  1.        ,
        -0.02639458,  0.02637076],
       [-0.23728171,  0.02631703,  0.16257918, ..., -0.02639458,
         1.        , -0.05771776],
       [ 0.24414068,  0.24682541,  0.30441038, ...,  0.02637076,
        -0.05771776,  1.        ]])